In [1]:
%pip install -r requirements.txt

  Using cached aiohappyeyeballs-2.4.3-py3-none-any.whl (14 kB)
  Using cached aiohttp-3.10.8-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
  Using cached anyio-4.6.0-py3-none-any.whl (89 kB)
  Using cached arxiv-2.1.3-py3-none-any.whl (11 kB)
  Using cached async_timeout-4.0.3-py3-none-any.whl (5.7 kB)
  Using cached attrs-24.2.0-py3-none-any.whl (63 kB)
  Using cached cachetools-5.5.0-py3-none-any.whl (9.5 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
  Using cached charset_normalizer-3.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (142 kB)
  Using cached click-8.1.7-py3-none-any.whl (97 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl (28 kB)
  Using cached duckduckgo_search-6.2.13-py3-none-any.whl (27 kB)
  Using cached faiss_cpu-1.8.0.post1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (

In [2]:
!mkdir data
#
! wget "https://arxiv.org/pdf/1810.04805.pdf" -O ./data/BERT_arxiv.pdf
! wget "https://arxiv.org/pdf/2005.11401" -O ./data/RAG_arxiv.pdf
! wget "https://arxiv.org/pdf/2310.11511" -O ./data/self_rag_arxiv.pdf
! wget "https://arxiv.org/pdf/2401.15884" -O ./data/crag_arxiv.pdf

--2024-10-04 14:41:44--  https://arxiv.org/pdf/1810.04805.pdf
Resolving arxiv.org (arxiv.org)... 151.101.131.42, 151.101.3.42, 151.101.67.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://arxiv.org/pdf/1810.04805 [following]
--2024-10-04 14:41:44--  http://arxiv.org/pdf/1810.04805
Connecting to arxiv.org (arxiv.org)|151.101.131.42|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775166 (757K) [application/pdf]
Saving to: ‘./data/BERT_arxiv.pdf’

./data/BERT_arxiv.p 100%[===================>] 757.00K  3.82MB/s    in 0.2s    

2024-10-04 14:41:44 (3.82 MB/s) - ‘./data/BERT_arxiv.pdf’ saved [775166/775166]

--2024-10-04 14:41:44--  https://arxiv.org/pdf/2005.11401
Resolving arxiv.org (arxiv.org)... 151.101.195.42, 151.101.67.42, 151.101.131.42, ...
Connecting to arxiv.org (arxiv.org)|151.101.195.42|:443... connected.
HTTP request sent, awaiting response... 200

Import Required Dependencies

In [5]:
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,SummaryIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool,QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters,FilterCondition
from typing import List,Optional

In [6]:
import  nest_asyncio
nest_asyncio.apply()

Read the Documents

In [7]:
documents = SimpleDirectoryReader(input_files = ['./data/self_rag_arxiv.pdf']).load_data()
print(len(documents))
print(f"Document Metadata: {documents[0].metadata}")

30
Document Metadata: {'page_label': '1', 'file_name': 'self_rag_arxiv.pdf', 'file_path': 'data/self_rag_arxiv.pdf', 'file_type': 'application/pdf', 'file_size': 1405127, 'creation_date': '2024-10-04', 'last_modified_date': '2023-10-19'}


Split the documents into chunks/nodes

In [8]:
splitter = SentenceSplitter(chunk_size=1024,chunk_overlap=100)
nodes = splitter.get_nodes_from_documents(documents)
print(f"Length of nodes : {len(nodes)}")
print(f"get the content for node 0 :{nodes[0].get_content(metadata_mode='all')}")

Length of nodes : 43
get the content for node 0 :page_label: 1
file_name: self_rag_arxiv.pdf
file_path: data/self_rag_arxiv.pdf
file_type: application/pdf
file_size: 1405127
creation_date: 2024-10-04
last_modified_date: 2023-10-19

Preprint.
SELF-RAG: LEARNING TO RETRIEVE , GENERATE ,AND
CRITIQUE THROUGH SELF-REFLECTION
Akari Asai†, Zeqiu Wu†, Yizhong Wang†§, Avirup Sil‡, Hannaneh Hajishirzi†§
†University of Washington§Allen Institute for AI‡IBM Research AI
{akari,zeqiuwu,yizhongw,hannaneh }@cs.washington.edu ,avi@us.ibm.com
ABSTRACT
Despite their remarkable capabilities, large language models (LLMs) often produce
responses containing factual inaccuracies due to their sole reliance on the paramet-
ric knowledge they encapsulate. Retrieval-Augmented Generation (RAG), an ad
hoc approach that augments LMs with retrieval of relevant knowledge, decreases
such issues. However, indiscriminately retrieving and incorporating a fixed number
of retrieved passages, regardless of whether retrieval 

Instantiate the vectorstore

In [9]:
import chromadb
db = chromadb.PersistentClient(path="./chroma_db_mistral")
chroma_collection = db.get_or_create_collection("multidocument-agent")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

Instantiate the embedding model

In [10]:
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")

Settings.embed_model = embed_model

Settings.chunk_size = 1024


/home/fx/Downloads/Projects/multi-agent-rag/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 5 files: 100%|██████████| 5/5 [00:31<00:00,  6.28s/it]


In [22]:
!source .env

In [42]:
import os 
from dotenv import load_dotenv
from llama_index.llms.mistralai import MistralAI

load_dotenv()
# mistral_api_key=  os.environ["MISTRAL_API_KEY"] 
llm = MistralAI(model="mistral-large-latest",api_key=os.getenv("MISTRAL_API_KEY"))

In [39]:
#instantiate Vectorstore
from llama_index.core.llms import MockLLM
name = "BERT_arxiv"
vector_index = VectorStoreIndex(nodes,storage_context=storage_context)
vector_index.storage_context.vector_store.persist(persist_path="/content/chroma_db")
#
# Define Vectorstore Autoretrieval tool
def vector_query(query:str,page_numbers:Optional[List[str]]=None)->str:
  '''
  perform vector search over index on
  query(str): query string needs to be embedded
  page_numbers(List[str]): list of page numbers to be retrieved,
                          leave blank if we want to perform a vector search over all pages
  '''
  page_numbers = page_numbers or []
  metadata_dict = [{"key":'page_label',"value":p} for p in page_numbers]
  #
  query_engine = vector_index.as_query_engine(llm=MockLLM(),similarity_top_k =2,
                                              filters = MetadataFilters.from_dicts(metadata_dict,
                                                                                    condition=FilterCondition.OR)
                                              )
  #
  response = query_engine.query(query)
  return response
#
#llamiondex FunctionTool wraps any python function we feed it
vector_query_tool = FunctionTool.from_defaults(name=f"vector_tool_{name}",
                                              fn=vector_query)
# Prepare Summary Tool
summary_index = SummaryIndex(nodes)
# summary_query_engine = summary_index.as_query_engine(llm=MockLLM(), embed_model="local")
summary_query_engine = summary_index.as_query_engine(llm=MockLLM(), response_mode="tree_summarize",
                                                      se_async=True,)
summary_query_tool = QueryEngineTool.from_defaults(name=f"summary_tool_{name}",
                                                    query_engine=summary_query_engine,
                                                  description=("Use ONLY IF you want to get a holistic summary of the documents."
                                              "DO NOT USE if you have specified questions over the documents."))

Add of existing embedding ID: 7ccbb8d8-e7c2-4bd8-8b94-3684ad6f67b6
Add of existing embedding ID: dbda8faa-81c9-4a27-8524-dd98c9633c9e
Add of existing embedding ID: 6e9c8032-2133-4a59-9a49-34a32f09d34a
Add of existing embedding ID: 93a60dcd-0bfc-4c2c-ab2b-d659b5065e53
Add of existing embedding ID: 4f803f2e-11df-4015-a92a-184ddda23666
Add of existing embedding ID: 2a08ab2c-4c54-463f-a5ff-7656008e9ec6
Add of existing embedding ID: d25218e9-ca55-4e3f-b076-05e66deeec91
Add of existing embedding ID: 3ba2fe0a-6525-4779-aa7d-14958a151376
Add of existing embedding ID: dacc0850-27aa-4a5e-827c-908fe6b1ec1c
Add of existing embedding ID: b4422c0d-15ef-4d24-942a-87381265674a
Add of existing embedding ID: fc9a841f-5241-4e32-9eef-275de9d0eef4
Add of existing embedding ID: 964436bd-7ccc-4345-a512-cd5a060dc757
Add of existing embedding ID: 4a2fd4cf-ad13-450a-ad34-1d8a0bc46430
Add of existing embedding ID: ad79ace7-53c5-4914-a41a-ea02cfd2a875
Add of existing embedding ID: e301813a-20da-42cc-9155-18b17805

Test the LLM

In [40]:
response = llm.predict_and_call([vector_query_tool],
                                "Summarize the content in page number 2",
                                verbose=True)
response

=== Calling Function ===
Calling function: vector_tool_BERT_arxiv with args: {"query": "Summarize the content in page number 2", "page_numbers": ["2"]}
=== Function Output ===
Context information is below.
---------------------
page_label: 2
file_path: data/self_rag_arxiv.pdf

Our analysis demonstrates the effectiveness of training and inference with
reflection tokens for overall performance improvements as well as test-time model customizations
(e.g., balancing the trade-off between citation previsions and completeness).
2 R ELATED WORK
Retrieval-Augmented Generation. Retrieval-Augmented Generation (RAG) augments the input
space of LMs with retrieved text passages (Guu et al., 2020; Lewis et al., 2020), leading to large
improvements in knowledge-intensive tasks after fine-tuning or used with off-the-shelf LMs (Ram
et al., 2023). A more recent work (Luo et al., 2023) instruction-tunes an LM with a fixed number
2

page_label: 2
file_path: data/self_rag_arxiv.pdf

Preprint.
Step 1: Retri

AgentChatResponse(response="Context information is below.\n---------------------\npage_label: 2\nfile_path: data/self_rag_arxiv.pdf\n\nOur analysis demonstrates the effectiveness of training and inference with\nreflection tokens for overall performance improvements as well as test-time model customizations\n(e.g., balancing the trade-off between citation previsions and completeness).\n2 R ELATED WORK\nRetrieval-Augmented Generation. Retrieval-Augmented Generation (RAG) augments the input\nspace of LMs with retrieved text passages (Guu et al., 2020; Lewis et al., 2020), leading to large\nimprovements in knowledge-intensive tasks after fine-tuning or used with off-the-shelf LMs (Ram\net al., 2023). A more recent work (Luo et al., 2023) instruction-tunes an LM with a fixed number\n2\n\npage_label: 2\nfile_path: data/self_rag_arxiv.pdf\n\nPreprint.\nStep 1: Retrieve K documentsCalifornia was named after a ﬁctional island in a Spanish book. Prompt How did US states get their names? \nUS sta